## Crear Dataset en el formato necesario

In [1]:
import os
import shutil

from_folder = "SoccerNet/calibration"
to_folder = "dataset-det/images"

# Crear la carpeta destino si no existe
os.makedirs(to_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(to_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    # Listar todas las imágenes en la subcarpeta
    images = [img for img in os.listdir(subfolder_path) if img.endswith(".jpg") or img.endswith(".png")]

    # Copiar las imágenes a la carpeta destino
    for img in images:
        src_path = os.path.join(subfolder_path, img)
        dst_path = os.path.join(to_folder_path, img)
        shutil.copy(src_path, dst_path)

print("Imágenes copiadas correctamente.")

Imágenes copiadas correctamente.


In [1]:
import os
import sys
import json
from_folder = "SoccerNet/calibration"
labels_folder = "dataset-det/labels"

# Diccionario para mapear etiquetas a IDs
class_mapping = {
    "Goal right crossbar": 0,
    "Goal right post left": 1,
    "Goal right post right": 2,
    "Goal left post left": 3,
    "Goal left post left ": 4,
    "Goal left post left": 5,
    "Goal left post right": 6,
    "Goal left crossbar": 7,
    "Small rect. right main": 8,
    "Small rect. right top": 9,
    "Side line right": 10,
    "Side line top": 11,
    "Side line bottom": 12,
    "Big rect. right top": 13,
    "Big rect. right main": 14,
    "Circle right": 15,
    "Side line left": 16,
    "Big rect. left top": 17,
    "Big rect. left bottom": 18,
    "Big rect. left main": 19,
    "Big rect. right bottom": 20,
    "Small rect. right bottom": 21,
    "Small rect. left top": 22,
    "Small rect. left bottom": 23,
    "Small rect. left main": 24,
    "Circle left": 25,
    "Circle central": 26,
    "Middle line": 27,
    "Line unknown": 28,
    "Goal unknown": 29,
}

# Función auxiliar para obtener el bounding box
def get_bbox_from_points(points):
    xs = [p["x"] for p in points]
    ys = [p["y"] for p in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min
    return x_center, y_center, width, height

# Crear la carpeta destino si no existe
os.makedirs(labels_folder, exist_ok=True)

# Iterar sobre cada subcarpeta
dirs = os.listdir(from_folder)
for dir_name in dirs:
    subfolder_path = os.path.join(from_folder, dir_name)
    to_folder_path = os.path.join(labels_folder, dir_name)
    if not os.path.isdir(subfolder_path):
        continue  # Saltar si no es carpeta
    
    if not os.path.exists(to_folder_path):
        os.makedirs(to_folder_path)

    #listar todos los json
    jsons = [json for json in os.listdir(subfolder_path) if json.endswith(".json")]

    for j in jsons:
        #si el nombre del archivo no es numérico, saltar
        if not j[:-5].isdigit():
            print(f"Nombre de archivo no numérico: {j}")
            continue
        with open(os.path.join(subfolder_path, j), "r") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error al cargar el archivo JSON {j}: {e}")
                continue
            label_txt = os.path.join(to_folder_path, j.replace(".json", ".txt"))
            with open(label_txt, "w") as label_file:
                for label, points in data.items():
                    if label not in class_mapping:
                        print(f"Etiqueta desconocida: {label} en {j}")
                    if label not in class_mapping or len(points) < 2:
                        continue
                    class_id = class_mapping[label]
                    x_c, y_c, w, h = get_bbox_from_points(points)
                    label_file.write(f"{class_id} {x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}\n")
print("Etiquetas generadas correctamente.")

Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Nombre de archivo no numérico: match_info.json
Etiquetas generadas correctamente.


# Finetunear Modelo

In [2]:
from ultralytics import YOLO

# Cargar un modelo base
model = YOLO('yolov8n.pt')  
# Entrenar (fine-tune) con tus datos
model.train(data='line_dataset_detail.yaml', epochs=50, imgsz=640, batch=16)

New https://pypi.org/project/ultralytics/8.3.142 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=line_dataset_detail.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, opt

train: Scanning /home/dani/Desktop/futbol/dataset-det/labels/train... 14513 images, 0 backgrounds, 379 corrupt: 100%|██████████| 14513/14513 [00:04<00:00, 3158.12it/s]

train: /home/dani/Desktop/futbol/dataset-det/images/train/00005.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000009]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00009.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000009]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00019.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000005]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00026.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000009]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00042.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000009]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00332.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.000014]
train: /home/dani/Desktop/futbol/dataset-det/images/train/00460.jpg: ignorin

train: New cache created: /home/dani/Desktop/futbol/dataset-det/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2372.7±1208.9 MB/s, size: 141.3 KB)


val: Scanning /home/dani/Desktop/futbol/dataset-det/labels/valid... 2796 images, 0 backgrounds, 82 corrupt: 100%|██████████| 2796/2796 [00:01<00:00, 2105.50it/s]

...

                   all       2714      26158      0.808      0.673      0.718      0.592



50 epochs completed in 1.610 hours.
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from /home/dani/Desktop/futbol/runs/detect/train6/weights/best.pt, 6.2MB

Validating /home/dani/Desktop/futbol/runs/detect/train6/weights/best.pt...
Ultralytics 8.3.141 🚀 Python-3.12.3 torch-2.7.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060, 7911MiB)
Model summary (fused): 72 layers, 3,011,498 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 85/85 [00:13<00:00,  6.48it/s]


                   all       2714      26158      0.808      0.673      0.718      0.592
   Goal right crossbar       1102       1102      0.829       0.79      0.818      0.681
  Goal right post left       1081       1081      0.621     0.0681      0.142     0.0531
 Goal right post right        982        982      0.588      0.113      0.213     0.0787
   Goal left post left       1009       1009      0.656      0.135      0.239     0.0958
  Goal left post right       1110       1110      0.613      0.084      0.169     0.0648
    Goal left crossbar       1156       1156      0.851      0.791      0.839      0.691
Small rect. right main       1220       1220      0.818      0.835      0.861      0.798
 Small rect. right top       1102       1102       0.83      0.753       0.81      0.551
       Side line right       1253       1253      0.836      0.871      0.897      0.851
         Side line top       1679       1679       0.88      0.765      0.832      0.619
      Side line botto

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x71d828b7e420>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0

In [3]:
#guardar el modelo
model.save("yolov8n_line_detail.pt")